In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled)


In [2]:
%matplotlib inline
import utils_modified; reload(utils_modified)
from utils_modified import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
import numpy as np
import random
import sys

In [4]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Embedding, Activation, LSTM, merge, Flatten, Dropout, Lambda
from keras.models import Model, Sequential
from keras.engine.topology import Merge
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers.convolutional import *
from keras.utils.data_utils import get_file

In [161]:
import quandl # pip install quandl
import pandas as pd

In [5]:
# https://keras.io/getting-started/sequential-model-guide/

In [156]:
def builder():
    # data array : 20days x 15stocks
    # note that we can name any layer by passing it a "name" argument.
    main_input = Input(shape=(20,15), dtype='float32', name='main_input')

    # a LSTM will transform the vector sequence into a single vector,
    # containing information about the entire sequence
    lstm_out = LSTM(32)(main_input)

    output = Dense(1, activation='relu')(lstm_out)

    model = Model(input=main_input, output=output)

    model.compile(optimizer='rmsprop', loss='mean_squared_error')
    
    return model

In [162]:
model1 = builder() # will be trained on simulated data

In [158]:
model1.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
main_input (InputLayer)          (None, 20, 15)        0                                            
____________________________________________________________________________________________________
lstm_4 (LSTM)                    (None, 32)            6144        main_input[0][0]                 
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 1)             33          lstm_4[0][0]                     
Total params: 6,177
Trainable params: 6,177
Non-trainable params: 0
____________________________________________________________________________________________________


In [159]:
if False:
    X = np.random.random((50,20,15))
    Y = np.random.random((50,1))
else:
    AllXs = []
    AllYs = []
    for i in range(100):
        t = (np.random.rand()-0.5)*6 # an offset for the sinus model
        Xs = []
        for stp in range(20):
            vol = 2 + math.sin(t+stp*0.2) # a market volatility
            Xs.append( np.random.randn(1,1,15)*vol ) # one slice of stock returns
        futurevol = 2 + math.sin(t+(20-1+5)*0.2)
        #print(vol, nextvol)
        AllXs.append( np.concatenate(Xs, axis=1) )
        AllYs.append( np.array([futurevol]).reshape((1,1)) )
    X = np.concatenate(AllXs, axis=0)
    Y = np.concatenate(AllYs, axis=0)

In [160]:
model1.fit(X, Y, batch_size=5, nb_epoch=20)

Epoch 1/20
100/100 [==============================] - 0s - loss: 3.6379     
Epoch 2/20
100/100 [==============================] - 0s - loss: 2.8001     
Epoch 3/20
100/100 [==============================] - 0s - loss: 1.7201     
Epoch 4/20
100/100 [==============================] - 0s - loss: 0.9740     
Epoch 5/20
100/100 [==============================] - 0s - loss: 0.7180     
Epoch 6/20
100/100 [==============================] - 0s - loss: 0.5716     
Epoch 7/20
100/100 [==============================] - 0s - loss: 0.4653     
Epoch 8/20
100/100 [==============================] - 0s - loss: 0.3708     
Epoch 9/20
100/100 [==============================] - 0s - loss: 0.2965     
Epoch 10/20
100/100 [==============================] - 0s - loss: 0.2360     
Epoch 11/20
100/100 [==============================] - 0s - loss: 0.1820     
Epoch 12/20
100/100 [==============================] - 0s - loss: 0.1337     
Epoch 13/20
100/100 [==============================] - 0s - loss: 0.1044 

In [86]:
def qData(tick='MSFT'):
    return quandl.get("WIKI/"+tick+".4",
                      start_date="2007-01-01",
                      end_date="2016-12-31",
                      collapse="daily")

In [87]:
TICKERS = ['MSFT','JPM','INTC','DOW','KO',
             'MCD','CAT','WMT','MMM','AXP',
             'BA','GE','XOM','PG','JNJ']

In [96]:
try:
    D.keys()
except:
    print('create empty Quandl cache')
    D = {}

for tckr in TICKERS:
    if not(tckr in D.keys()):
        print(tckr)
        qdt = qData(tckr)
        qdt.rename(columns={'Close': tckr}, inplace = True)
        D[tckr] = qdt
        
for tck in D.keys():
    assert(D[tck].keys() == [tck])

In [97]:
for tck in D.keys():
    print(D[tck].shape)

(2517, 1)
(2502, 1)
(2518, 1)
(2505, 1)
(2518, 1)
(2518, 1)
(2518, 1)
(2518, 1)
(2518, 1)
(2518, 1)
(2518, 1)
(2518, 1)
(2503, 1)
(2512, 1)
(2518, 1)


In [98]:
J = D[TICKERS[0]].join(D[TICKERS[1]])
for tck in TICKERS[2:]:
    J = J.join(D[tck])

In [99]:
J.head(5)

,MSFT,JPM,INTC,DOW,KO,MCD,CAT,WMT,MMM,AXP,BA,GE,XOM,PG,JNJ
Date,,,,,,,,,,,,,,,
2007-01-03,29.86,48.07,20.35,40.27,48.58,43.87,61.16,47.55,78.26,60.36,89.17,37.97,74.11,64.54,66.40
2007-01-04,29.81,48.19,21.17,40.09,48.60,43.54,61.00,47.78,77.95,59.92,89.53,37.75,72.72,64.05,67.23
2007-01-05,29.64,47.79,21.10,39.71,48.26,43.54,60.22,47.39,77.42,59.13,89.15,37.56,73.24,63.50,66.62
2007-01-08,29.93,47.95,21.01,39.49,48.57,43.71,60.29,47.00,77.59,59.69,88.94,37.55,72.65,63.64,66.51
2007-01-09,29.96,47.75,21.03,39.39,48.61,43.79,60.62,47.39,77.68,59.31,88.00,37.55,72.09,63.48,66.26


In [102]:
J.isnull().sum()

MSFT     0
JPM      0
INTC     0
DOW      6
KO       0
MCD      0
CAT      0
WMT     15
MMM      0
AXP      1
BA      13
GE       0
XOM     16
PG       0
JNJ      0
dtype: int64

In [165]:
J2 = J.fillna(method='ffill')
#J2[J['WMT'].isnull()]

In [166]:
LogDiffJ = J2.apply(np.log).diff(periods=1, axis=0)
LogDiffJ.drop(LogDiffJ.index[0:1], inplace=True)
LogDiffJ.shape

(2517, 15)

In [167]:
MktData = LogDiffJ.as_matrix(columns=None) # as numpy.array
MktData.shape

(2517, 15)

In [284]:
model2 = builder() # will be trained on market data

In [298]:
if True:
    AllXs = []
    AllYs = []
    for i in range(100):
        t = np.random.randint(50, MktData.shape[0]-100) # an offset for whole historics
        Xs = []
        for stp in range(20):
            Xs.append( MktData[t+stp,:].reshape(1,1,-1)*100 ) # one slice of stock returns
        futurevol = math.sqrt(np.sum(MktData[t+20:t+20+10,:]*MktData[t+20:t+20+10,:]))*100
        #print(futurevol)
        AllXs.append( np.concatenate(Xs, axis=1) )
        AllYs.append( np.array([futurevol]).reshape((1,1)) )
    X = np.concatenate(AllXs, axis=0)
    Y = np.concatenate(AllYs, axis=0)

In [299]:
X.shape, Y.shape

((100, 20, 15), (100, 1))

In [300]:
print(np.min(X), np.mean(X), np.max(X))
print(np.min(Y), np.mean(Y), np.max(Y))

-23.3016325713 0.0293632244999 22.3917160238
6.67103515354 19.9754065213 70.1243805881


In [283]:
# WARNING : need to scale data to speed up training !!!
factorX = 0.10
factorY = 0.03

## WARNING :  Need a **VALIDATION** test too

In [285]:
model2.fit(factorX*X, factorY*Y, batch_size=25, nb_epoch=200) # seems like 150 epochs are required to converge : BUG ?

Epoch 1/200
100/100 [==============================] - 0s - loss: 0.5564     
Epoch 2/200
100/100 [==============================] - 0s - loss: 0.4165     
Epoch 3/200
100/100 [==============================] - 0s - loss: 0.2714     
Epoch 4/200
100/100 [==============================] - 0s - loss: 0.1632     
Epoch 5/200
100/100 [==============================] - 0s - loss: 0.1328     
Epoch 6/200
100/100 [==============================] - 0s - loss: 0.1227     
Epoch 7/200
100/100 [==============================] - 0s - loss: 0.1120     
Epoch 8/200
100/100 [==============================] - 0s - loss: 0.1001     
Epoch 9/200
100/100 [==============================] - 0s - loss: 0.0911     
Epoch 10/200
100/100 [==============================] - 0s - loss: 0.0973     
Epoch 11/200
100/100 [==============================] - 0s - loss: 0.0830     
Epoch 12/200
100/100 [==============================] - 0s - loss: 0.0841     
Epoch 13/200
100/100 [==============================] - 0s - 